In [13]:
import pandas as pd
import pybaseball as pyb
import bar_chart_race as bcr
import warnings
pd.set_option('display.max_columns', None)
warnings.simplefilter('ignore')

In [50]:
df = pyb.statcast(start_dt='2022-04-07', end_dt='2022-10-06')

This is a large query, it may take a moment to complete


100%|█████████████████████████████████████████| 183/183 [03:48<00:00,  1.25s/it]


In [51]:
df = df[df['events']=='home_run']

In [52]:
data = df.copy()

In [53]:
data['Date'] = pd.to_datetime(data['game_date']) - pd.to_timedelta(7, unit='d')
data = data.groupby(['batter', pd.Grouper(key='Date', freq='W-MON')])['events'].count().reset_index().sort_values('Date')


In [54]:
player_ids = data['batter'].unique()
pid = pyb.playerid_reverse_lookup(player_ids, key_type='mlbam')

In [39]:
pid[pid['name_last']=='judge']

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
285,judge,aaron,592450,judga001,judgeaa01,15640,2016.0,2022.0


In [55]:
data = data.merge(pid[['name_last','name_first','key_mlbam']],left_on='batter',right_on='key_mlbam',how='left')

In [56]:
data['Player'] = data['name_first'] + ' ' +data['name_last']

In [57]:
data.groupby('Player')['events'].sum().sort_values(ascending=False).head(20)

Player
aaron judge          62
kyle schwarber       46
pete alonso          40
mike trout           40
austin riley         38
yordan alvarez       37
christian walker     36
paul goldschmidt     35
mookie betts         35
rowdy tellez         35
matt olson           34
shohei ohtani        34
anthony santander    33
corey seager         33
vladimir guerrero    32
anthony rizzo        32
manny machado        32
willy adames         31
eugenio suarez       31
giancarlo stanton    31
Name: events, dtype: int64

In [64]:
df_weekly = data.pivot_table(values = 'events',index=['Date'],columns='Player')

In [66]:
df_weekly.fillna(0, inplace=True)

In [70]:
df_weekly.iloc[:, 0:-1] = df_weekly.iloc[:, 0:-1].cumsum()

In [71]:
top_hr = set()
for index, row in df_weekly.iterrows():
    top_hr |= set(row[row > 0].sort_values(ascending=False).head(10).index)
df_weekly = df_weekly[top_hr]
df_weekly.index = 'Week ' + df_weekly.index.astype(str)
df_weekly.to_csv('HomeRun_Bar_Plot.csv')

In [72]:
df_weekly

Player,byron buxton,christian walker,paul goldschmidt,mitch haniger,jesus sanchez,ozzie albies,george springer,salvador perez,aaron judge,seiya suzuki,tommy edman,rowdy tellez,mike trout,brad miller,connor joe,marcell ozuna,francisco lindor,jose ramirez,mookie betts,bryce harper,shohei ohtani,c. j. cron,corey seager,jean segura,hunter renfroe,pete alonso,willy adames,danny jansen,joc pederson,yordan alvarez,kyle schwarber,vladimir guerrero,seth brown,austin riley,giancarlo stanton,anthony rizzo,nolan arenado,ty france
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Week 2022-04-04,3.0,1.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,2.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0
Week 2022-04-11,3.0,2.0,0.0,3.0,2.0,5.0,2.0,2.0,1.0,4.0,3.0,1.0,2.0,2.0,2.0,4.0,3.0,3.0,0.0,2.0,3.0,5.0,1.0,2.0,0.0,3.0,1.0,2.0,2.0,3.0,2.0,5.0,2.0,2.0,2.0,3.0,4.0,2.0
Week 2022-04-18,6.0,4.0,0.0,3.0,3.0,6.0,3.0,5.0,3.0,4.0,3.0,3.0,4.0,2.0,4.0,4.0,4.0,4.0,2.0,3.0,3.0,6.0,1.0,2.0,2.0,3.0,2.0,2.0,6.0,4.0,4.0,5.0,2.0,4.0,2.0,5.0,5.0,5.0
Week 2022-04-25,7.0,5.0,2.0,3.0,3.0,6.0,6.0,5.0,8.0,4.0,3.0,4.0,6.0,2.0,4.0,4.0,4.0,7.0,3.0,4.0,4.0,8.0,4.0,2.0,5.0,4.0,5.0,2.0,6.0,7.0,7.0,6.0,2.0,7.0,4.0,9.0,6.0,5.0
Week 2022-05-02,9.0,6.0,2.0,3.0,3.0,6.0,6.0,5.0,9.0,4.0,3.0,7.0,7.0,3.0,4.0,4.0,5.0,7.0,5.0,6.0,6.0,9.0,4.0,4.0,6.0,7.0,8.0,2.0,6.0,8.0,7.0,7.0,2.0,7.0,6.0,9.0,7.0,5.0
Week 2022-05-09,11.0,7.0,4.0,3.0,3.0,6.0,7.0,6.0,12.0,4.0,4.0,8.0,9.0,5.0,4.0,5.0,6.0,8.0,7.0,9.0,8.0,9.0,7.0,6.0,9.0,8.0,9.0,3.0,7.0,11.0,9.0,7.0,3.0,7.0,10.0,10.0,8.0,5.0
Week 2022-05-16,11.0,11.0,7.0,3.0,4.0,6.0,8.0,6.0,17.0,4.0,4.0,9.0,12.0,6.0,4.0,8.0,6.0,10.0,10.0,9.0,9.0,11.0,8.0,6.0,9.0,11.0,9.0,3.0,7.0,12.0,10.0,7.0,4.0,8.0,11.0,10.0,9.0,6.0
Week 2022-05-23,11.0,12.0,11.0,3.0,5.0,6.0,9.0,6.0,18.0,4.0,4.0,10.0,13.0,7.0,4.0,10.0,8.0,13.0,15.0,10.0,11.0,13.0,10.0,6.0,9.0,13.0,9.0,6.0,11.0,14.0,11.0,9.0,5.0,12.0,11.0,11.0,9.0,7.0
Week 2022-05-30,12.0,14.0,12.0,3.0,7.0,6.0,11.0,8.0,21.0,4.0,5.0,10.0,13.0,7.0,4.0,10.0,9.0,14.0,16.0,13.0,11.0,14.0,11.0,6.0,9.0,16.0,9.0,7.0,13.0,16.0,14.0,13.0,5.0,14.0,11.0,13.0,10.0,8.0


In [8]:
df = nfl.import_pbp_data(years=[2021])
df_weekly = df.groupby(['rusher_id','rusher','week'])['yards_gained'].sum().reset_index()
df_weekly = df_weekly[df_weekly['week'] < 19]
df_weekly = df_weekly.pivot_table(values = 'yards_gained',index = ['week'], columns = 'rusher')
df_weekly.fillna(0, inplace=True)
df_weekly.iloc[:, 0:-1] = df_weekly.iloc[:, 0:-1].cumsum()
top_rushers = set()
for index, row in df_weekly.iterrows():
    top_rushers |= set(row[row > 0].sort_values(ascending=False).head(10).index)
df_weekly = df_weekly[top_rushers]
df_weekly.index = 'Week ' + df_weekly.index.astype(str)
df_weekly.to_csv('Rushing Bar Plot.csv')

2021 done.
Downcasting floats.


In [73]:
bcr.bar_chart_race(df=df_weekly, 
                   n_bars=15, 
                   sort='desc',
                   title='HOME RUN LEADERS BY WEEK',
                   period_length=1000,
                   interpolate_period=False,
                   steps_per_period=10,
                   fixed_max=True)